In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import datetime
import time
import pandas as pd
import numpy as np
from tqdm import tqdm

import re

In [ ]:
acc = 150 # 인터넷,it,통신,모바일,게임 

In [ ]:
# 원티드, 잡플래닛, 점핏을 사용함에 있어서 해당 code cell은 사용하지 않을 예정
incruit_occu_list = ['웹프로그래머', '응용프로그래머', '시스템프로그래머', '데이터베이스', '데이터베이스·DBA', '네트워크·서버·보안', 'ERP·시스템분석·설계',
            '컨텐츠·사이트운영', '웹디자인', '통신·모바일', '소프트웨어·하드웨어', '전산·IT기술지원', '멀티미디어(동영상·음향)', '게임(Game)',
            '빅데이터·AI(인공지능)', 'HTML·퍼블리싱·UI개발']

incruit_occu_code = [574, 375, 579, 405, 390, 435, 420, 598, 410, 530, 578, 400, 415, 535, 632, 633]

In [ ]:
url = "https://job.incruit.com/jobdb_list/searchjob.asp?cate=occu"

def get_job_list(url, acc):

    job_list = []

    driver.get(url)
    button_open = driver.find_element(By.XPATH, f'//*[@id="occ1_list_{acc}"]') #인터넷It통신모바일게임 직종 선택
    button_open.click()
    driver.implicitly_wait(2)

    # 직종 클릭 및 초기화버튼은 없어도 스크래핑하는데 문제는 없음 따라서 나중에 필요하다면 없애고 코드를 간결화 할 수 있음
    # 현 단계에서는 잘 돌아가는지 확인하기 위해 클릭 단계를 진행
    i = 0
    for code in incruit_occu_code:
        i += 1

        button_open = driver.find_element(By.XPATH, f'//*[@id="occ2_list_{acc}_{code}"]') #세부직종 선택
        button_open.click()
        driver.implicitly_wait(3)

        # 해당 세부직종 코드 안의 직무 개수 반환
        job_middle_class = driver.find_element(By.XPATH, f'//*[@id="occ2_list_{acc}_{code}"]').get_attribute('data-item-name')
        tmp = driver.find_elements(By.ID, f'occ3_ul_{code}') 
        job_num = len(tmp[0].text.split('\n'))
        
        for n in range(2, job_num+1):
            job = driver.find_element(By.XPATH, f'/html/body/div[2]/div[1]/div/div[1]/div/div/div/div[2]/div[1]/div[2]/div[4]/ul/li[{n}]/label')
            driver.implicitly_wait(3)
    
            job_name = re.sub(r' \(\d+\)', '', job.text) # 직무명, (개수)는 없앰
            recruit_cnt = int(re.sub(r"[\(, \)]", '', job.text.split(' ')[-1]))
            job_code = int(job.get_attribute('data-item').replace('occ3_', '')) #직무코드

            job_list.append((job_middle_class, job_name, job_code, recruit_cnt))


        if i == 8: #인크루트는 직종 선택을 10개를 초과하지 못하게 되어있기 때문에 그 전에 초기화를 눌러서 해소
            button_open = driver.find_element(By.XPATH, '//*[@id="incruit_contents"]/div[1]/div/div/div/div[3]/div[1]/button[1]')
            button_open.click()
            driver.implicitly_wait(2)

            i = 0
    
    return pd.DataFrame(job_list, columns = ['직업중분류', '직업소분류', 'code', '공고수'])



In [ ]:
#창 띄우기

chrome_options = Options()
chrome_options.add_experimental_option("detach", True) #창을 보면서 작업할 수 있음
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
# driver.maximize_window()

In [ ]:
jl = get_job_list(url, 150)
# 총 직무수 305개

In [ ]:
jl.to_csv('C:/datafile/incruit_개발직군.csv', encoding = 'utf-8-sig', index = False)